In [3]:
from farm.utils import initialize_device_settings
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor, SquadProcessor
from farm.data_handler.data_silo import DataSilo
from farm.eval import Evaluator
from farm.modeling.adaptive_model import AdaptiveModel
from pathlib import Path
from farm.infer import Inferencer



In [4]:
import pandas as pd
df = pd.read_csv("/home/graphn/repositories/you_conscious/dl_xp/data/colour/train_color.tsv",sep="\t")
label_list_1 = df["label"].tolist()
label_list_1 = list(set(label_list_1))
df = pd.read_csv("/home/graphn/repositories/you_conscious/dl_xp/data/colour/test_color.tsv",sep="\t")
label_list_2 = df["label"].tolist()
label_list_2 = list(set(label_list_2))
label_list = label_list_1 + label_list_2 + [""]
label_list = list(set(label_list_1+label_list_2)) 
texts = df["text"].tolist()
y_true  = df["label"].tolist()

In [5]:
batch_size = 40




In [6]:
# 9. Load it & harvest your fruits (Inference)
save_dir =  "/home/graphn/repositories/you_conscious/dl_xp/trained_models/color_distil"
basic_texts = []
for t in texts:
    basic_texts.append({"text":t})
print(len(basic_texts))
model = Inferencer.load(save_dir, gpu=True, return_class_probs=False,task_type="text_classification")
result = model.inference_from_dicts(dicts=basic_texts)

04/16/2021 17:51:26 - INFO - farm.utils -   Using device: CUDA 
04/16/2021 17:51:26 - INFO - farm.utils -   Number of GPUs: 1
04/16/2021 17:51:26 - INFO - farm.utils -   Distributed Training: False
04/16/2021 17:51:26 - INFO - farm.utils -   Automatic Mixed Precision: None
04/16/2021 17:51:26 - INFO - farm.modeling.language_model -   
04/16/2021 17:51:26 - INFO - farm.modeling.language_model -   LOADING MODEL
04/16/2021 17:51:26 - INFO - farm.modeling.language_model -   =============
04/16/2021 17:51:26 - INFO - farm.modeling.language_model -   Model found locally at /home/graphn/repositories/you_conscious/dl_xp/trained_models/color_distil


853


04/16/2021 17:51:27 - INFO - farm.modeling.language_model -   Loaded /home/graphn/repositories/you_conscious/dl_xp/trained_models/color_distil
04/16/2021 17:51:27 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
04/16/2021 17:51:27 - WARNING - farm.modeling.prediction_head -   `layer_dims` will be deprecated in future releases
04/16/2021 17:51:27 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 25]
04/16/2021 17:51:27 - INFO - farm.modeling.prediction_head -   Loading prediction head from /home/graphn/repositories/you_conscious/dl_xp/trained_models/color_distil/prediction_head_0.bin
04/16/2021 17:51:29 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'DistilBertTokenizerFast'
04/16/2021 17:51:29 - INFO - farm.data_handler.processor -   Initialized processor without tasks. Supply `metric` and `label_list` to the constructor for using the default task or add a custom task later via processor.add_ta

04/16/2021 17:51:29 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
04/16/2021 17:51:29 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
04/16/2021 17:51:29 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
04/16/2021 17:51:29 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
04/16/2021 17:51:29 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
04/16/2021 17:51:29 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
04/16/2021 17:51:30 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
Inferencing Samples:   0%|          | 0/4 [00:00<?, ? Batches/s]04/16/2021 17:51:30 - WARNING - farm.dat

Inferencing Samples: 100%|██████████| 2/2 [00:00<00:00, 49.86 Batches/s]


In [7]:
import numpy as np
predictions=[]
i = 0
for batch in result:
    for x in batch["predictions"]:
        if True:
            prediction = x["label"]
            predictions.append(prediction)
        #probabilities = x["probability"]
        #position = np.argmax(probabilities)
        #print(id_2_labels[position],y_true[i])
        #labels = list(id_2_labels.values())
        #predictions.append({label:prob for label,prob in zip(probabilities,labels)})
        i+=1

In [8]:
predictions[0:2]

["['Rot']", "['Gelb']"]

In [9]:
from sklearn.metrics import classification_report
def evaluate(predictions,Y_test):
    return classification_report(predictions, Y_test,output_dict=True)#, target_names=target_names

In [10]:
evaluation=evaluate(predictions,y_true)

/home/graphn/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/graphn/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/graphn/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/graphn/.local/lib/python3.8/site-packages/

In [11]:
precisions = []
recalls = []
f1s = []
supports = []
labels = []
for k,v in evaluation.items():
    if type(v) != float: #!="accuracy":
        labels.append(k)
        precisions.append(v["precision"])
        recalls.append(v["recall"])
        f1s.append(v["f1-score"])
        supports.append(v["support"])
x = pd.DataFrame({"labels":labels,"precision":precisions,"recall":recalls,"f1_score":f1s,"support":supports})
x.to_csv("color_v1_distil.tsv",sep="\t")